# Functions in Cadabra
Cadabra is implemented in C++, however builds on the Python language -- as a result, python syntax is completely supported. 

We write Cadabra functions just as we would Python functions:

In [1]:
def truncate(poly, n):
    x^{a}::Weight(label=\epsilon).
    
    ans = Ex("0")
    
    for i in range(0, n+1):
        foo := @(poly).
        bah = Ex(f"\epsilon = {i}")
        keep_weight(foo, bah)
    
        ans = ans + foo
    return ans

And similiarly call them just as we would in Python:

In [2]:
quartic := c^{a}
    + c^{a}_{b} x^b
    + c^{a}_{b c} x^b x^c
    + c^{a}_{b c d} x^b x^c x^d
    + c^{a}_{b c d e} x^b x^c x^d x^e;

truncate(quartic, 2);

${}c^{a}+c^{a}\,_{b} x^{b}+c^{a}\,_{b c} x^{b} x^{c}+c^{a}\,_{b c d} x^{b} x^{c} x^{d}+c^{a}\,_{b c d e} x^{b} x^{c} x^{d} x^{e}$

${}c^{a}+c^{a}\,_{b} x^{b}+c^{a}\,_{b c} x^{b} x^{c}$

# Zooming and tags
When working on an expression, you may wish to at times apply a substitution or simplficiation only to a given section of the whole. To facilitate this, Cadabra provides the notion of *zooms* and *tags*.

## `zoom` and `unzoom`
Say we wanted to replace the second $v^a$ with a $w^a$ in the expression

$$
A_{a} v^{a} + B_{a} v^{a} + C_{a} v^{a}.
$$

To do this, we use `zoom` and `unzoom`:

In [3]:
expr := A_{a} v^{a} + B_{a} v^{a} + C_{a} v^{a};

zoom(expr, $B_{a} Q??$);
substitute(expr, $v^{a} -> w^{a}$);
unzoom(expr);

${}A_{a} v^{a}+B_{a} v^{a}+C_{a} v^{a}$

${} \ldots +B_{a} v^{a}+ \ldots $

${} \ldots +B_{a} w^{a}+ \ldots $

${}A_{a} v^{a}+B_{a} w^{a}+C_{a} v^{a}$

We use the generic pattern match `Q??`, which will match any arbitrary expression composed of sums and/or products of arbitrary tensors.

## Tagging 
As a somewhat contrived example, let us consider an anti-symmetric tensor $V_{ab}$ and work on the expression

In [4]:
expr := 2 V_{a b} - 3 V_{b a}; 

${}2V_{a b}-3V_{b a}$

We could assign the `::AntiSymmetric` property to $V_{ab}$ and use `canonicalise`, however for illustrative purposes, let us solve this using the notion of `tag`. We define two functions:

In [5]:
def add_tags(obj, tag):
    n = 0
    ans = Ex("0")
    for i in obj.top().terms():
        foo = obj[i]
        bah = Ex(tag + "_{" + str(n) + "}")
        ans := @(ans) + @(bah)@(foo).
        n += 1
    return ans
    
def clear_tags(obj, tag):
    ans := @(obj).
    foo = Ex(tag + "_{a?} -> 1")
    substitute(ans, foo)
    return ans

And use them as such:

In [6]:
expr = add_tags(expr, '\\mu');

zoom(expr, $\mu_{1} Q??$);
substitute(expr, $V_{a b} -> - V_{b a}$);
unzoom(expr);

expr = clear_tags(expr,'\\mu');

${}2\mu_{0} V_{a b}-3\mu_{1} V_{b a}$

${} \ldots -3\mu_{1} V_{b a}$

${} \ldots +3\mu_{1} V_{a b}$

${}2\mu_{0} V_{a b}+3\mu_{1} V_{a b}$

${}5V_{a b}$